In [1]:
#!/usr/bin/env python
# -=-<[ Bismillahirrahmanirrahim ]>-=-
# -*- coding: utf-8 -*-
# @Date    : 2022-03-12 18:02:42
# @Author  : Dahir Muhammad Dahir (dahirmuhammad3@gmail.com)
# @Link    : link
# @Version : 1.0.0


import angr, monkeyhex

project = angr.Project('/bin/true')
state = project.factory.entry_state()



WARNING | 2022-03-12 18:04:30,058 | cle.loader | The main binary is a position-independent executable. It is being loaded with a base address of 0x400000.


In [2]:
one = state.solver.BVV(1, 64)
one


<BV64 0x1>

In [3]:
one_hundred = state.solver.BVV(100, 64)
one_hundred

<BV64 0x64>

In [4]:
weird_nine = state.solver.BVV(9, 27)
weird_nine

<BV27 0x9>

In [5]:
one + one_hundred

<BV64 0x65>

In [6]:
one_hundred + 0x100

<BV64 0x164>

In [7]:
one_hundred - one * 200

<BV64 0xffffffffffffff9c>

In [8]:
weird_nine.zero_extend(64 - 27)

<BV64 0x9>

In [10]:
one + weird_nine.zero_extend(64 - 27)

<BV64 0xa>

In [11]:
x = state.solver.BVS('x', 64)
x

<BV64 x_35_64>

In [12]:
y = state.solver.BVS('y', 64)
y

<BV64 y_36_64>

In [13]:
x + one

<BV64 x_35_64 + 0x1>

In [14]:
(x + one) / 2


<BV64 (x_35_64 + 0x1) / 0x2>

In [15]:
x - y

<BV64 x_35_64 - y_36_64>

In [16]:
tree = (x + 1) / (y + 2)
tree

<BV64 (x_35_64 + 0x1) / (y_36_64 + 0x2)>

In [17]:
tree.op

'__floordiv__'

In [18]:
tree.args

(<BV64 x_35_64 + 0x1>, <BV64 y_36_64 + 0x2>)

In [19]:
tree.args[0].op

'__add__'

In [20]:
tree.args[0].args

(<BV64 x_35_64>, <BV64 0x1>)

In [21]:
tree.args[0].args[1].op 

'BVV'

In [22]:
tree.args[0].args[1].args

(0x1, 0x40)

In [23]:
x == 1

<Bool x_35_64 == 0x1>

In [24]:
x == one

<Bool x_35_64 == 0x1>

In [25]:
x > 2

<Bool x_35_64 > 0x2>

In [26]:
x + y == one_hundred + 5

<Bool x_35_64 + y_36_64 == 0x69>

In [27]:
one_hundred

<BV64 0x64>

In [28]:
one_hundred > 5

<Bool True>

In [29]:
one_hundred > -5

<Bool False>

In [30]:
one_hundred.SGT(-5)

<Bool True>

In [31]:
yes = one == 1
no = one == 2
maybe = x == y
state.solver.is_true(yes)

True

In [32]:
state.solver.is_false(yes)

False

In [33]:
state.solver.is_true(no)

False

In [34]:
state.solver.is_false(no)

True

In [35]:
state.solver.is_true(maybe)

False

In [36]:
state.solver.is_true(maybe)

False

In [37]:
x, y

(<BV64 x_35_64>, <BV64 y_36_64>)

In [43]:
state.solver.add(x > y)
state.solver.add(y > 4)
state.solver.add(8 > x)
state.solver.eval(x)
state.solver.eval(y)

0x5

In [44]:
state = project.factory.entry_state()
input = state.solver.BVS('input', 64)
operation = (((input + 4) * 3) >> 1) + input
output = 200
state.solver.add(operation == output)
state.solver.eval(input)

0x3333333333333381

In [45]:
state.solver.add(input < 2 ** 32)
state.satisfiable()


False

In [46]:
state = project.factory.entry_state()
state.solver.add(x - y >= 4)
state.solver.add(y > 0)
state.solver.eval(x)


0x3

In [47]:
state.solver.eval(y)

0xffffffffffffffff

In [48]:
state.solver.eval(x + y)

0x2

In [49]:
state = project.factory.entry_state()
a = state.solver.FPV(3.2, state.solver.fp.FSORT_DOUBLE)
a


<FP64 FPV(3.2, DOUBLE)>

In [50]:
b = state.solver.FPS('b', state.solver.fp.FSORT_DOUBLE)
b


<FP64 FPS(FP_b_38_64, DOUBLE)>

In [51]:
a + b


<FP64 fpAdd(RM.RM_NearestTiesEven, FPV(3.2, DOUBLE), FPS(FP_b_38_64, DOUBLE))>

In [52]:
a + 4.4


<FP64 FPV(7.6000000000000005, DOUBLE)>

In [53]:
b + 2 < 0

<Bool fpLT(fpAdd(RM.RM_NearestTiesEven, FPS(FP_b_38_64, DOUBLE), FPV(2.0, DOUBLE)), FPV(0.0, DOUBLE))>

In [54]:
state.solver.add(b + 2 < 0)
state.solver.add(b + 2 > -1)
state.solver.eval(b)


-2.0001978874206543

In [55]:
a.raw_to_bv()


<BV64 0x400999999999999a>

In [56]:
a

<FP64 FPV(3.2, DOUBLE)>

In [57]:
b.raw_to_bv()


<BV64 fpToIEEEBV(FPS(FP_b_38_64, DOUBLE))>

In [58]:
state.solver.BVV(0, 64).raw_to_fp()


<FP64 FPV(0.0, DOUBLE)>

In [59]:
state.solver.BVS('x', 64).raw_to_fp()

<FP64 fpToFP(x_39_64, DOUBLE)>

In [60]:
a


<FP64 FPV(3.2, DOUBLE)>

In [61]:
a.val_to_bv(12)

<BV12 0x3>

In [62]:
a.val_to_bv(12).val_to_fp(state.solver.fp.FSORT_FLOAT)


<FP32 FPV(3.0, FLOAT)>

In [63]:
state.solver.eval(state.solver.BVV(0x41424344, 32), cast_to=bytes)

b'ABCD'